In [ ]:
import datetime
import matplotlib.dates as mdates
from matplotlib import pyplot as plt

from summer.utils import ref_times_to_dti

from autumn.settings import Region
from autumn.tools.project import get_project, TimeSeriesSet
from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print
from autumn.tools.inputs.covid_au.queries import get_both_vacc_coverage
from autumn.models.covid_19.strat_processing.vaccination import find_vacc_strata

### Specify what we're going to run

In [ ]:
region = Region.VICTORIA
project = get_project("covid_19", region)
targets_path = f"../../../autumn/projects/covid_19/victoria/victoria/targets.secret.json"
ts_set = TimeSeriesSet.from_file(targets_path)
start_time = ref_times_to_dti(REF_DATE, [project.param_set.baseline["time"]["start"]])[0]

In [ ]:
new_params = {}

In [ ]:
project.param_set.baseline = project.param_set.baseline.update(new_params)

### Run the model

In [ ]:
model = project.run_baseline_model(project.param_set.baseline)
derived_df = model.get_derived_outputs_df()

### Specify the outputs

In [ ]:
outputs = (
    "notifications", "infection_deaths", "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"
)

### Plot the outputs

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 15), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)
for i_ax, axis in enumerate(axes.reshape(-1)):
    output_name = outputs[i_ax]
    output = ts_set.get(output_name)
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.plot(derived_df.index, derived_df[outputs[i_ax]])
    axis.scatter(ref_times_to_dti(REF_DATE, output.times), output.values, c="k")
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(output_name)
    axis.set_xlim(left=start_time)

### Inspect the computed value processes around contact tracing

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)

derived_df["prop_all_traced"] = derived_df["prop_contacts_with_detected_index"] * derived_df["prop_detected_traced"]

computed_processes = ("prevalence", "prop_contacts_with_detected_index", "prop_detected_traced", "prop_all_traced")
for i_ax, axis in enumerate(axes.reshape(-1)):
    process = computed_processes[i_ax]
    axis.plot(derived_df.index, derived_df[process])
    axis.set_title(process)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=start_time)
    if i_ax > 0:
        axis.set_ylim((0., 1.))

### Vaccination stratification
Should check that this is the parameter configuration being requested

In [ ]:
vacc_strata, _ = find_vacc_strata(True, True, True)

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(12, 8))

# Loop over the vaccination types being implemented in the model
lower_value = [0.] * len(derived_df)
for stratum in vacc_strata[::-1]:
    working_value = derived_df[f"proportion_{stratum}"]
    upper_value = lower_value + working_value
    axis.fill_between(derived_df.index, lower_value, upper_value, label=stratum)
    lower_value = upper_value

vacc_times, vacc_coverage = get_both_vacc_coverage(region.upper(), start_age=0.)
vacc_dates = ref_times_to_dti(BASE_DATETIME, [int(i) for i in vacc_times])  # Converting numpy ints returned
lagged_dates = ref_times_to_dti(BASE_DATETIME, [int(i) + 14 for i in vacc_times])
axis.plot(vacc_dates, vacc_coverage, color="k", label="actual vaccination")
axis.plot(lagged_dates, vacc_coverage, color="k", linestyle="--", label="lagged for immunity")
axis.tick_params(axis="x", labelrotation=45)
axis.set_title(f"{region} vaccination check")
axis.legend()